<center><font size="10">Mehta Purvil</font></center>
<br><center><font size="6">Github Username: purvilmehta06</font></center>
<br><center><font size="6">USC ID: 6104797766</font></center>

# Package imports

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from scipy.stats import bootstrap
warnings.simplefilter('ignore')

# 1. Time Series Classification Part 1: Feature Creation/Extraction

## (a) Get the AReM Data Set and cleaning

In [2]:
data_folder_path = '../data/AReM/'
files = {}
for folder in os.listdir(data_folder_path):
    if folder == '.DS_Store': continue
    for file in os.listdir(data_folder_path + folder):
        key = data_folder_path + folder + '/' + file
        files[key] = pd.read_csv(key, skiprows = 4, usecols = [0, 1, 2, 3, 4, 5, 6])
        del files[key][files[key].columns[0]]

#### Note: Datasets with problems were bending2/dataset4.csv having space as delimeter instead of comma, and cycling/dataset9.csv having an extra comma in the end of file, were cleaned manually.

## (b) Test and Train Data

In [3]:
test_data = {}
train_data = {}
for folder in os.listdir(data_folder_path):
    if folder == '.DS_Store': continue
    for file in os.listdir(data_folder_path + folder):
        file_name = data_folder_path + folder + '/' + file
        if (folder == 'bending1' or folder == 'bending2'):
            if file == 'dataset1.csv' or file == 'dataset2.csv':
                test_data[file_name] = files[file_name]
            else:
                train_data[file_name] = files[file_name]
        else:
            if file == 'dataset1.csv' or file == 'dataset2.csv' or file == 'dataset3.csv':
                test_data[file_name] = files[file_name]
            else:
                train_data[file_name] = files[file_name]

print("Total Train Dataset Files: ", len(train_data))
print("Total Test Dataset Files: ", len(test_data))

Total Train Dataset Files:  69
Total Test Dataset Files:  19


## (c) Feature Extraction

### i. Research

In time series classification, time-domain features are extracted from the raw data to capture the underlying patterns and characteristics of the time series. Some of the commonly used time-domain features in time series classification are:

* Mean: The arithmetic mean of the values in the time series.
* Standard deviation: A measure of the variability or spread of the time series values.
* Skewness: A measure of the asymmetry of the distribution of the time series values.
* Variance: A measure of the dispersion of the time series values around the mean.
* Interquartile range: The difference between the 75th and 25th percentiles of the time series values.
* Cross Correlation - Cross-correlation is another commonly used time-domain feature in time series classification. It measures the similarity between two time series by computing the correlation between them at different time lags. Cross-correlation can help identify patterns that are similar between two time series, even if they are not aligned in time. 
* Autocorrelation: Autocorrelation is a measure of the correlation between a time series and a lagged version of itself, and can help identify repeating patterns or cycles within the time series. 
* Autoregressive (AR) - These models are a type of time series model that use past values of a time series to predict future values. The order of the AR model, denoted by p, specifies how many past values of the time series are used in the model.
* Other parameters of ARIMA (Autoregressive integrated moving average) model like Integrated (I) and Moving Average (MA) are also some of the important features of any Time series data.

These are just some examples of the many time-domain features that can be used in time series classification. The specific choice of features may depend on the nature of the data and the particular classification problem at hand.

### ii. Extraction

In [12]:
cols = ['min', 'max', 'mean', 'median', 'std', '1st quartile', '3rd quartile']

columns = []
for j in range(6):
    for i in range(len(cols)):
        columns.append(cols[i] + str(j+1))
    
parameters = ['min', 'max', 'mean', '50%', 'std', '25%', '75%']
extract_data = []
for k, v in files.items():
    stats = files[k].describe()
    temp = []
    for feature in list(files[k].columns):
        for i, parameter in enumerate(parameters):
            temp.append(stats[feature][parameter])
    extract_data.append(temp)
extract_data = pd.DataFrame(extract_data, columns = columns)
extract_data

,min1,max1,mean1,median1,std1,1st quartile1,3rd quartile1,min2,max2,mean2,...,std5,1st quartile5,3rd quartile5,min6,max6,mean6,median6,std6,1st quartile6,3rd quartile6
0,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.8300
1,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700
2,33.00,47.75,42.179812,43.50,3.670666,39.1500,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000
3,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000
4,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,3.328104,...,3.120057,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600
84,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,2.374208,...,2.890347,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000
85,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,2.736021,...,3.617702,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175
86,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,2.420083,...,2.851673,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300


### iii. Standard Deviation

In [13]:
std_extract_data = []
stds = []
for col in list(extract_data.columns):
    stds.append(extract_data[col].std())
    std_extract_data.append([col, stds[-1]])
std_extract_data = pd.DataFrame(std_extract_data, columns = ['Feature', 'Standard Deviation'])
std_extract_data

,Feature,Standard Deviation
0,min1,9.569975
1,max1,4.394362
2,mean1,5.335718
3,median1,5.440054
4,std1,1.772153
5,1st quartile1,6.153590
6,3rd quartile1,5.138925
7,min2,0.000000
8,max2,5.062729
9,mean2,1.574164


In [14]:
confidence_interval = []
for i, col in enumerate(list(extract_data.columns)):
    bootResult = bootstrap((extract_data[col], ), np.std, confidence_level = 0.9, 
                              random_state = 1, method = 'percentile')

    confidence_interval.append([col, stds[i], bootResult.confidence_interval.low, 
                                bootResult.confidence_interval.high])

confidence_interval = pd.DataFrame(confidence_interval, 
                                   columns = ['Feature Name', 'Standard Deviation', 'Lower Bound', 'Upper Bound'])
confidence_interval

,Feature Name,Standard Deviation,Lower Bound,Upper Bound
0,min1,9.569975,8.217348,10.704231
1,max1,4.394362,3.306005,5.269063
2,mean1,5.335718,4.675214,5.837115
3,median1,5.440054,4.760294,5.964649
4,std1,1.772153,1.562763,1.935570
5,1st quartile1,6.153590,5.542312,6.603819
6,3rd quartile1,5.138925,4.308947,5.806972
7,min2,0.000000,0.000000,0.000000
8,max2,5.062729,4.595134,5.377385
9,mean2,1.574164,1.393100,1.698068


### iv. Select Features

1. **Mean**: Considering many outliers, mean is one of the good measures to identify central (i.e. common) tendency when the data distribution is continuous.
2. **Median**: Since the data has too much outliers, and outliers can change the overall mean value of the data towards them, median could be the good feature of central (i.e. common) tendency that is less affected by outliers.
3. **Standard Deviation**: According to the defination, standard deviation is a measure to identify datapoints that are away from the mean. It also help us identify the underlying data distribution. 

# 2. ISLR 3.7.4

## (a) Linear Train

**Cubic regression would overfit the data if the actual relationship between X and Y is linear.** The training error for the cubic regression model would be less than the regular linear regression because of the overfitting. However, the test error would be higher. **Thus, cubic regression would have a lesser RSS value than linear regression.**

## (b) Linear Test

Because of the overfitting happening with the cubic regression model, **testing RSS for cubic regression would be higher than the regular linear regression model.**

## (c) Not Linear Train

**Linear regression would underfit the data if the actual relationship between X and Y is not linear.** Because of the underfitting (in other words, high bias - less variance), **training error of cubic regression (training RSS) would be lesser than the regular linear regression.** 

## (d) Not Linear Testing

**The information provided is insufficient** as the answer will depend on the data amd how far from linear it is distributed